In [1]:
import json
import pandas as pd

#读取json
with open('./config/config.json','r+') as file:
    content=file.read()
config=json.loads(content)

In [2]:
def judge(fea_data,config,way):
    rate=0
    config_dict=config[way]
    fea_list=list(config_dict.keys())
    for fea in fea_list:
        value=fea_data[fea]
        bins=config_dict[fea]["bins"]
        rates = config_dict[fea]["rate"]

        if value<min(bins):
            rate=rate+rates[0]
        elif value>max(bins):
            rate=rate+rates[-1]
        else:
            for b in range(len(bins)-1):
                if value>bins[:-1][b] and value<=bins[1:][b]:
                    rate=rate+rates[b]
    return rate/len(fea_list)


##### 开始运行

In [3]:
tit='N_all.csv'
tit='P_all.csv'
# tit='P_ice.csv'
# tit='P_her.csv'
# tit='P_mix.csv'
# tit='P_ice1.csv'
tit="N-2-6.csv"
tit='n.csv'
# tit='all.csv'

In [4]:
path = 'E:\\desktop\\data\\app\\fea\\'
path = 'E:\\desktop\\data\\app\\223\\fea\\ok\\'
#读取数据
data = pd.read_csv(path+tit,encoding='gbk')
data = data.set_index('name')

result={'name':[],'all':[],'opi':[],'cns':[],'other':[]}
name_list=list(data.index)

youla={"累加>2":[],"(opi+cns+other>1.5) and all>0.5":[],"(opi or cns or other) and all":[],"(opi or cns) and all":[],"四选二":[]}
for i in range(len(data)):
    row=data.iloc[i]
    result['name'].append(name_list[i])
    fea_data=dict(row)

    rate=[]
    for way in ('all','opi','cns','other'):
        juedge_rate=judge(fea_data,config,way)
        rate.append(juedge_rate)
        result[way].append(round(juedge_rate,2))

    #累加>2
    if sum(rate)>2:
        youla["累加>2"].append(1)
    else:
        youla["累加>2"].append(0)

    #all>0.5 and opi+cns+other>1.5
    if rate[0]>0.5 and sum(rate[1:])>1.5:
        youla["(opi+cns+other>1.5) and all>0.5"].append(1)
    else:
        youla["(opi+cns+other>1.5) and all>0.5"].append(0)

    ret=[]
    for j in rate:
        if j>0.5:
            ret.append(1)
        else:
            ret.append(0)

    if sum(ret[:3])>=1 and ret[0]==1:
        youla["(opi or cns or other) and all"].append(1)
    else:
        youla["(opi or cns or other) and all"].append(0)

    if sum(ret[:2])>=1 and ret[0]==1:
        youla["(opi or cns) and all"].append(1)
    else:
        youla["(opi or cns) and all"].append(0)

    if sum(ret)>=2:
        youla["四选二"].append(1)
    else:
        youla["四选二"].append(0)


result.update(youla)

result=pd.DataFrame(result)
result=result.set_index('name')

psum=len(result)
for key in result.columns:
    p=sum(result[key])
    print(key,p,'/',psum,p/psum)

result.to_csv(tit,encoding="gbk")

all 156.18999999999997 / 521 0.29978886756238
opi 116.45000000000005 / 521 0.22351247600767762
cns 171.01999999999995 / 521 0.32825335892514385
other 165.52000000000027 / 521 0.31769673704414636
累加>2 46 / 521 0.08829174664107485
(opi+cns+other>1.5) and all>0.5 38 / 521 0.07293666026871401
(opi or cns or other) and all 52 / 521 0.09980806142034548
(opi or cns) and all 52 / 521 0.09980806142034548
四选二 48 / 521 0.09213051823416507
